In [1]:
# Changing directory to access the database
import os
os.chdir("/Users/komangandikawirasantosa/Face-Rec/DeepFace-IPYNB")

In [2]:
import cv2
import numpy as np
import os
from insightface import app
from scipy.spatial.distance import cosine

# Load the InsightFace model
face_recognition = app.FaceAnalysis()
face_recognition.prepare(ctx_id=0, det_size=(640, 640))

# Load the face database
database_path = 'cropped_faces'  # Replace with the path to your database
face_embeddings = {}
for person_name in os.listdir(database_path):
    person_folder = os.path.join(database_path, person_name)
    if os.path.isdir(person_folder):
        face_embeddings[person_name] = []
        for img_name in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                faces = face_recognition.get(img)
                for face in faces:
                    # Store the normalized embedding of the first face detected
                    face_embeddings[person_name].append(face.normed_embedding)

/Users/komangandikawirasantosa/opt/anaconda3/lib/python3.9/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/Users/komangandikawirasantosa/opt/anaconda3/lib/python3.9/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CoreMLExecutionProvider, AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.in

In [3]:
# Load the target image for comparison
target_image_path = 'Test1.jpeg'  # Replace with the path to your target image
target_image = cv2.imread(target_image_path)

if target_image is None:
    print("Error: Could not read target image.")
    exit()

# Perform face detection and recognition on the target image
faces = face_recognition.get(target_image)

# Display results
for face in faces:
    # Get bounding box and landmarks
    bbox = face.bbox.astype(int)
    identity = face.normed_embedding

    # Compare with the database
    recognized_person = None
    min_distance = float('inf')
    for person_name, embeddings in face_embeddings.items():
        for embedding in embeddings:
            distance = cosine(identity, embedding)
            if distance < min_distance:
                min_distance = distance
                recognized_person = person_name

    # Draw bounding box and label on the image
    if recognized_person is not None and min_distance < 0.5:  # Adjust threshold as necessary
        label = recognized_person
    else:
        label = "Unknown"

    cv2.rectangle(target_image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
    cv2.putText(target_image, label, (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Show the resulting image
cv2.imshow('Face Recognition', target_image)
cv2.waitKey(0)  # Wait until a key is pressed
cv2.destroyAllWindows()

/Users/komangandikawirasantosa/opt/anaconda3/lib/python3.9/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


## Second Version

In [1]:
# Changing directory to access the database
import os
os.chdir("/Users/komangandikawirasantosa/Face-Rec/DeepFace-IPYNB")

In [2]:
import insightface
import cv2
import numpy as np
import os
import pickle
from scipy.spatial.distance import cosine

# Load the InsightFace model
model = insightface.app.FaceAnalysis(providers=['CPUExecutionProvider'])
model.prepare(ctx_id=0)

# Directory to store embeddings
embedding_dir = "face_embeddings_v2"

def create_and_store_embeddings(image_dir):
    """
    Generate and store embeddings for each person.
    """
    if not os.path.exists(embedding_dir):
        os.makedirs(embedding_dir)

    for person_name in os.listdir(image_dir):
        person_dir = os.path.join(image_dir, person_name)

        if not os.path.isdir(person_dir):
            continue

        embeddings = []

        # Process each image of the person
        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            img = cv2.imread(image_path)
            if img is None:
                print(f"Warning: Unable to read image {image_path}. Skipping.")
                continue

            # Resize the image to a larger size (e.g., 112x112 or 224x224)
            img = cv2.resize(img, (112, 112))  # Adjust size as needed

            # Get the embedding directly from the image
            try:
                face = model.get(img)  # This should return embeddings directly
                if len(face) > 0:
                    embeddings.append(face[0].normed_embedding)  # Get the embedding of the first face
                else:
                    print(f"No faces found in {image_path}.")
            except Exception as e:
                print(f"Error processing {image_path}: {e}")

        if embeddings:
            with open(os.path.join(embedding_dir, f"{person_name}.pkl"), "wb") as f:
                pickle.dump(embeddings, f)
            print(f"Saved embeddings for {person_name}")
        else:
            print(f"No embeddings found for {person_name}. Skipping.")

def load_embeddings():
    """
    Load stored embeddings of each person for recognition.
    """
    known_embeddings = {}
    for file_name in os.listdir(embedding_dir):
        person_name = file_name.split(".pkl")[0]
        with open(os.path.join(embedding_dir, file_name), "rb") as f:
            known_embeddings[person_name] = pickle.load(f)
    return known_embeddings

def recognize_faces(test_image_path, output_path, threshold=0.5):
    """
    Recognize faces in the test image, annotate them with name and distance, and save the result.
    """
    known_embeddings = load_embeddings()

    # Load the test image
    img = cv2.imread(test_image_path)
    faces = model.get(img)

    # Iterate over each detected face
    for face in faces:
        # Use normed_embedding
        embedding = face.normed_embedding
        min_distance = float("inf")
        identity = "Unknown"

        # Compare embedding with known embeddings
        for person_name, person_embeddings in known_embeddings.items():
            for stored_embedding in person_embeddings:
                # Compute cosine distance
                distance = cosine(embedding, stored_embedding)

                if distance < min_distance and distance < threshold:
                    min_distance = distance
                    identity = person_name

        # Draw bounding box, name, and distance
        box = face.bbox.astype(int)
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        
        # Display identity and distance
        label = f"{identity} ({min_distance:.2f})"
        cv2.putText(img, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    # Save the result image
    cv2.imwrite(output_path, img)
    print(f"Result saved to {output_path}")

# Example usage
# create_and_store_embeddings("path_to_your_cropped_images")
# recognize_faces("path_to_test_image.jpg", "output_image.jpg")

/Users/komangandikawirasantosa/opt/anaconda3/lib/python3.9/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.in

In [3]:
# Example Usage
# Step 1: Generate embeddings
create_and_store_embeddings("Database")  # Folder with one subfolder per person, each with images of that person

/Users/komangandikawirasantosa/opt/anaconda3/lib/python3.9/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


No faces found in Database/Tania/Tania.jpg.
Saved embeddings for Tania
No faces found in Database/Safira-Sama/Saf1.png.
Saved embeddings for Safira-Sama
Saved embeddings for Deci
Saved embeddings for Elen
No faces found in Database/Rachma/Rach3.png.
No faces found in Database/Rachma/Rach2.png.
No faces found in Database/Rachma/Rachma.jpg.
Saved embeddings for Rachma
No faces found in Database/Andika/Andi2.JPG.
Saved embeddings for Andika
No faces found in Database/Patricia/Patricia.jpg.
Saved embeddings for Patricia
Saved embeddings for Jumaga
Saved embeddings for BangRizal
No faces found in Database/Maul-Unyu/Maulana.jpg.
No faces found in Database/Maul-Unyu/Maul4.png.
Saved embeddings for Maul-Unyu
No faces found in Database/Rahmi/Rahmi.jpg.
No faces found in Database/Rahmi/Rahmi2.png.
No faces found in Database/Rahmi/Rahmi1.png.
No embeddings found for Rahmi. Skipping.
Saved embeddings for Devan
Saved embeddings for SamSulek


In [4]:
recognize_faces("Test1.jpeg", "output1.jpg")

Result saved to output1.jpg


In [5]:
recognize_faces("Test2.jpeg", "output2.jpg")

Result saved to output2.jpg


In [7]:
recognize_faces("Test3.jpg", "output3.jpg")

Result saved to output3.jpg


In [8]:
recognize_faces("Test4.jpg", "output4.jpg")

Result saved to output4.jpg


## Third Version??? ;(


pls no more